# СТАРТ

## Импорты и вводные данные

In [79]:
import requests
import datetime
import time
import csv

from openpyxl import Workbook
from openpyxl.utils.dataframe import dataframe_to_rows
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = (15, 5)

In [80]:
TOKEN_ID = '' # ← Поставь тут твой токен
API_V = '5.126'

In [81]:
def folower_count_get(owner_id):
    try:
        rf = requests.get('https://api.vk.com/method/groups.getMembers', params={'group_id': owner_id,
                                                                                 'v': API_V,
                                                                                 'access_token': TOKEN_ID})
    except Exception as err:
        return err
    
    rf_response = rf.json()
    return rf_response['response']['count']

In [82]:
# Читаем файл, который получили в блокноте с функциями парсинга
xcl_df = pd.read_csv('vk_wall_get_1public.csv', header=None)
xcl_df.columns=['N', 'id', 'from_id', 'owner_id', 'date', 'text', 'views', 'likes', 'reposts', 'comments']
xcl_df['link'] = 'https://vk.com/wall' + xcl_df['owner_id'].map(str) + '_' + xcl_df['id'].map(str)

# Конвертируем дату
xcl_df['date'] = pd.to_datetime(xcl_df['date'], unit='s')

# Считаем и добавляем столбцы ER.v и ER.p
xcl_df['ER.v'] = 100 * (xcl_df['likes'] + xcl_df['likes'] + xcl_df['comments']) / xcl_df['views']
xcl_df['ER.p'] = 100 * (xcl_df['likes'] + xcl_df['likes'] + xcl_df['comments']) / followers[-1]

xcl_df

,N,id,from_id,owner_id,date,text,views,likes,reposts,comments,link,ER.v,ER.p
0,0,397203,-1,-1,2020-12-07 08:45:00,10 декабря в 19:00 собираемся на седьмом митап...,10288,47,0,40,https://vk.com/wall-1_397203,1.302488,0.035062
1,1,397107,-1,-1,2020-12-03 15:55:12,Прокачали messages.pin: отныне бот с правами а...,12201,131,3,69,https://vk.com/wall-1_397107,2.712892,0.086609
2,2,397065,-1,-1,2020-12-02 08:56:06,Теперь можно размещать свой проект сразу на дв...,6637,35,0,40,https://vk.com/wall-1_397065,1.657375,0.028782
3,3,396954,-1,-1,2020-11-13 13:34:17,"Делимся некоторыми изменениями, связанными с C...",13572,72,2,29,https://vk.com/wall-1_396954,1.274683,0.045267
4,4,396931,-1,-1,2020-11-12 12:48:35,Выведите заработок на новый уровень — на 130% ...,340186,376,2,13,https://vk.com/wall-1_396931,0.224877,0.200169
...,...,...,...,...,...,...,...,...,...,...,...,...,...
176,176,240436,-1,-1,2011-02-05 11:54:32,Теперь openApi приложения тоже могут использов...,0,145,5,31,https://vk.com/wall-1_240436,inf,0.083992
177,177,238413,-1,-1,2011-02-04 15:40:53,Подключение виджетов в IFrame-приложениях http...,0,103,7,5,https://vk.com/wall-1_238413,inf,0.055210
178,178,236533,-1,-1,2011-02-03 18:17:06,Обновлён метод wall.post для публикации записе...,0,116,7,9,https://vk.com/wall-1_236533,inf,0.063060
179,179,55229,-1,-1,2010-11-08 20:13:17,Результаты второго этапа конкурса на разработк...,0,686,9,27,https://vk.com/wall-1_55229,inf,0.366060


In [83]:
xcl_df['owner_id'].unique()

array([-1])

In [84]:
#Соберём количество подписчиков в пабликах, которые анализируем, для рассчёта ER Post и других метрик:
followers = {owner_id: folower_count_get(abs(owner_id)) for owner_id in xcl_df['owner_id'].unique()}
followers

{-1: 382181}

## Стартуем Воркбук (будет файлом)

In [85]:
wb = Workbook() # Инициализация объекта Воркбук
wb

## Эксперименты (старый материал, пробы, можно не смотреть)

In [18]:
ws = wb.active
print(ws)

<Worksheet "Sheet">


In [28]:
ws = wb.create_sheet("Mysheet")
ws

<Worksheet "Mysheet4">

In [27]:
for row in ws.iter_rows(min_row=1, max_col=3, max_row=2):
    for cell in row:
        cell = cell
        print(cell)

<Cell 'Mysheet3'.A1>
<Cell 'Mysheet3'.B1>
<Cell 'Mysheet3'.C1>
<Cell 'Mysheet3'.A2>
<Cell 'Mysheet3'.B2>
<Cell 'Mysheet3'.C2>


In [31]:
for row in ws.values:
    for value in row:
        print(value)

In [33]:
for row in ws.iter_rows(min_row=1, max_col=3, max_row=2, values_only=True):
    print(row)

(None, None, None)
(None, None, None)


In [34]:
wb.save('excelstudy.xlsx')

**Ага**, если несколько раз выполнять команду `ws = wb.create_sheet("Mysheet")` то прошлый объект "Mysheet" никуда не денется из wb, и более того, в воркбук добавится новый объект "Mysheet2..N" (новый лист внутри файла), просто переменная ws будет указывать на новый лист, сохранив старый в файле.

In [35]:
wb.sheetnames

['Sheet', 'Mysheet', 'Mysheet1', 'Mysheet2', 'Mysheet3', 'Mysheet4']

In [36]:
wb = Workbook()

In [37]:
wb

In [40]:
from openpyxl.utils import get_column_letter

dest_filename = 'test_book.xlsx'

ws1 = wb.active
ws1.title = "range names"

for row in range(1, 40):
    ws1.append(range(600))

ws2 = wb.create_sheet(title="Pi")

ws2['F5'] = 3.14

ws3 = wb.create_sheet(title="Data")
for row in range(10, 20):
    for col in range(27, 54):
        ws3.cell(column=col, row=row, value="{0}".format(get_column_letter(col)))
        
print(ws3['AA10'].value)

wb.save(filename = dest_filename)


AA


In [42]:
ws1.values

<generator object Worksheet.values at 0x7f71ebcb7eb0>

In [18]:
# Эксперименты
#из файла Url	Likes	Reposts	Comments	Views	Date	Author	ER.v	ER.p
xcl_df['ER.v'] = 100 * (xcl_df['likes'] + xcl_df['likes'] + xcl_df['comments']) / xcl_df['views']
xcl_df['ER.p'] = 100 * (xcl_df['likes'] + xcl_df['likes'] + xcl_df['comments']) / followers[-1]


In [19]:
# Эксперименты
xcl_df['ER.v']

0      1.302488
1      2.712892
2      1.657375
3      1.274683
4      0.224877
         ...   
176         inf
177         inf
178         inf
179         inf
180         inf
Name: ER.v, Length: 181, dtype: float64

In [20]:
# Эксперименты
xcl_df['ER.p']

0      0.035062
1      0.086609
2      0.028782
3      0.045267
4      0.200169
         ...   
176    0.083992
177    0.055210
178    0.063060
179    0.366060
180    0.661734
Name: ER.p, Length: 181, dtype: float64

In [24]:
# Эксперименты
xcl_df[['link', 'likes', 'reposts', 'comments', 'views', 'date', 'from_id', 'ER.v', 'ER.p']]

,link,likes,reposts,comments,views,date,from_id,ER.v,ER.p
0,https://vk.com/wall-1_397203,47,0,40,10288,2020-12-07 08:45:00,-1,1.302488,0.035062
1,https://vk.com/wall-1_397107,131,3,69,12201,2020-12-03 15:55:12,-1,2.712892,0.086609
2,https://vk.com/wall-1_397065,35,0,40,6637,2020-12-02 08:56:06,-1,1.657375,0.028782
3,https://vk.com/wall-1_396954,72,2,29,13572,2020-11-13 13:34:17,-1,1.274683,0.045267
4,https://vk.com/wall-1_396931,376,2,13,340186,2020-11-12 12:48:35,-1,0.224877,0.200169
...,...,...,...,...,...,...,...,...,...
176,https://vk.com/wall-1_240436,145,5,31,0,2011-02-05 11:54:32,-1,inf,0.083992
177,https://vk.com/wall-1_238413,103,7,5,0,2011-02-04 15:40:53,-1,inf,0.055210
178,https://vk.com/wall-1_236533,116,7,9,0,2011-02-03 18:17:06,-1,inf,0.063060
179,https://vk.com/wall-1_55229,686,9,27,0,2010-11-08 20:13:17,-1,inf,0.366060


In [ ]:
# Эксперименты
# Разбиваем дату на два столбца если надо
pre_df = xcl_df['date'].str.split(' ',expand=True) # Делим дату по пробелу
pre_df.columns=['day','time']  # Формируем новые столбцы
for column in pre_df.columns:
    pre_df[column] = pre_df[column].str.replace(column+'=','') # Заменяем значение в каждой ячейке нового столбца на нужное
pre_df = pd.concat([xcl_df,pre_df],axis=1).drop('date',axis=1) # Контатенируем датафреймы, чтобы добавились созданные столбцы 
ndf = pre_df.drop(['N', 'id'], axis=1) # Удаляем ненужные уже два столбца
ndf['day'] =  pd.to_datetime(ndf['day'], format='%Y-%m-%d')
ndf

In [30]:
# Эксперименты
srez = xcl_df[xcl_df['date'].between('2020-11-01','2020-11-30', inclusive=True)]
srez

,N,id,from_id,owner_id,date,text,views,likes,reposts,comments,link,ER.v,ER.p
3,3,396954,-1,-1,2020-11-13 13:34:17,"Делимся некоторыми изменениями, связанными с C...",13572,72,2,29,https://vk.com/wall-1_396954,1.274683,0.045267
4,4,396931,-1,-1,2020-11-12 12:48:35,Выведите заработок на новый уровень — на 130% ...,340186,376,2,13,https://vk.com/wall-1_396931,0.224877,0.200169
5,5,396877,-1,-1,2020-11-10 15:20:00,Открыли методы и события для работы с VK Donut...,15147,109,7,41,https://vk.com/wall-1_396877,1.709910,0.067769
6,6,396765,-1,-1,2020-11-02 16:01:01,Обновили метод [https://vk.com/dev/messages.se...,16891,131,1,54,https://vk.com/wall-1_396765,1.870819,0.082684


In [27]:
# Эксперименты
srez['likes'].sum()

688

In [31]:
# Эксперименты
srez['reposts'].mean()

3.0

# Первый лист - данные по постам и ЕР

**xcl_df** - датафрейм с общими данными по всем собранным пабликам  
**wb** - воркбук в котором работаем (объект, который будет итоговым файлом)  
**ws1..N** - воркшит (объект, который будет страницей в файле)  
  
Чтобы записывать единственный воркшит в файл - лучше применить **.to_excel()** (Пандас)  
Чтобы записывать несколько воркшитов в файл - следует применять **.ExcelWriter()** (Пандас)  

In [86]:
ws1 = wb.active # Активируем объект Воркбук и создаем первый воркшит
ws1.title = "Посты" # Меняем имя воркшита, который создаётся по дефолту

In [87]:
# Формируем датафрейм по листу
ws1_xcl_df = xcl_df[['link', 'likes', 'reposts', 'comments', 'views', 'date', 'from_id', 'ER.v', 'ER.p']]

# Второй лист - Сводные данные по паблику

In [88]:
ws2 = wb.create_sheet(title="Сводные показатели")

In [89]:
PERIOD = ['2020-11-01', '2020-11-30'] # Задаём интервал сведения показателей ['от', 'до'], константа, вынести в начало
srez = xcl_df[xcl_df['date'].between(PERIOD[0], PERIOD[1], inclusive=True)]
srez

,N,id,from_id,owner_id,date,text,views,likes,reposts,comments,link,ER.v,ER.p
3,3,396954,-1,-1,2020-11-13 13:34:17,"Делимся некоторыми изменениями, связанными с C...",13572,72,2,29,https://vk.com/wall-1_396954,1.274683,0.045267
4,4,396931,-1,-1,2020-11-12 12:48:35,Выведите заработок на новый уровень — на 130% ...,340186,376,2,13,https://vk.com/wall-1_396931,0.224877,0.200169
5,5,396877,-1,-1,2020-11-10 15:20:00,Открыли методы и события для работы с VK Donut...,15147,109,7,41,https://vk.com/wall-1_396877,1.709910,0.067769
6,6,396765,-1,-1,2020-11-02 16:01:01,Обновили метод [https://vk.com/dev/messages.se...,16891,131,1,54,https://vk.com/wall-1_396765,1.870819,0.082684


In [90]:
# Формируем датафрейм по листу
# xcl_df[['link', 'likes', 'reposts', 'comments', 'views', 'date', 'from_id', 'ER.v', 'ER.p']]
ws2_xcl_df = pd.DataFrame()

link = 'vk.com/public1' # пока руками зададим, потом циклом
period = PERIOD[0] + ' ' + PERIOD[1]

ws2_xcl_df['link'] = link
ws2_xcl_df['period'] = period
ws2_xcl_df['period2'] = PERIOD
ws2_xcl_df['followers'] = followers[-1] # пока руками зададим, потом циклом
ws2_xcl_df['likes'] = srez['likes'].sum()
ws2_xcl_df['reposts'] = srez['reposts'].sum()
ws2_xcl_df['comments'] = srez['comments'].sum()
ws2_xcl_df['ER.v'] = srez['ER.v'].mean()
ws2_xcl_df['ER.p'] = srez['ER.p'].mean()

0    NaN
1    NaN
Name: period, dtype: object

# Третий лист - Сводные по неделям (не готово)

In [ ]:
#ws3 = wb.create_sheet(title="Сводные недельные")

ws3_xcl_df = pd.DataFrame() # Формируем датафрейм по листу

# Четвертый лист - Сводные по месяцам (не готово)

In [ ]:
#ws4 = wb.create_sheet(title="Сводные месячные")

ws4_xcl_df = pd.DataFrame() # Формируем датафрейм по листу

# Пятый лист - Графики (не готово)

In [ ]:
#ws5 = wb.create_sheet(title="Сводные графики")

ws5_xcl_df = pd.DataFrame() # Формируем датафрейм по листу

# Сохраняем

In [91]:
filename_path = 'itog_test_book.xlsx' # Тут надо имя файла сделать по имени паблика и периода
wb.save(filename = filename_path)

with pd.ExcelWriter(filename_path) as writer:
    ws1_xcl_df.to_excel(writer, sheet_name='Посты')
    ws2_xcl_df.to_excel(writer, sheet_name='Сводные показатели')
    #ws3_xcl_df.to_excel(writer, sheet_name='Сводные недельные')
    #ws4_xcl_df.to_excel(writer, sheet_name='Сводные месячные')
    #ws5_xcl_df.to_excel(writer, sheet_name='Сводные графики')

# Эксперименты

Тут пока ничего